# 0.Packages


In [4]:
%pip install ipython-autotime
%pip install anthropic==0.40.0
%pip install openai==1.57.4
%pip install aisuite
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.55.3
    Uninstalling openai-1.55.3:
      Successfully uninstalled openai-1.55.3
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 13.2 s (started: 2024-12-17 04:02:13 +00:00)


# 1.LLM


## System prompt


In [2]:
import json
import os

TEMPERATURE = 0.75
MAX_TOKENS = 2048

context_list = [
    {'role': 'system',
     'content': (
         'You are an assistant specialized in interpreting divination '
         'results. Use the following [Context] to provide insights and '
         'advice to users based on their questions. Always format your '
         'responses according to the [Response Format] below, ensuring '
         'personalized interpretations and actionable advice.\n'
         '\n'
         '[Response Format]\n '
         '您抽到的籤是: [Include 籤名 here]\n '
         '籤詩: [Include full 籤詩 here]\n '
         '\n'
         '籤詩解讀:\n '
         '小廟公建議:\n '
         '\n'
         '\n'
         '[Context] 籤名: 【中籤】中中 第五十八首-羅隱歸咎越王\n'
         '\n'
         '\n'
         '~籤詩~\n'
         '忠言善語君須記     \n'
         '莫向他方求別藝     \n'
         '勸君安守舊生涯     \n'
         '除卻有餘都不是     \n'
         '\n'
         '此籤有"虛心求教"之意。告誡當事人，勿恃才傲物。有些人因為本身才氣縱橫而自視甚高，因此目空一切，對於身邊的人事物，也常看不上眼，甚至語多譏諷、一派自以為是，非不得已，決不向他人求教。須知過於剛硬的個性會帶給別人不悅的感受而容易導致處處碰壁，如果能夠正視並收斂此種狂放自傲的個性，凡事虛心下問，加上本身的才華與能力，相信成就一定不只於此。對於身邊親朋好友的勸告，務必聽進心裡去，想一想別人為什麽這麽說。\n'
         '\n'
         '小廟公建議可參考聖意如下, 不需要全數給求籤者, 若有任何難或不利的訊號, 則建議小心慎重, 正向訊號則可達成!\n'
         '關於家宅: 平常\n'
         '關於自身: 不利\n'
         '關於求財: 吉\n'
         '關於交易: 成\n'
         '關於婚姻: 阻\n'
         '關於六甲: 驚\n'
         '關於行人: 遲\n'
         '關於田蠶: 五分\n'
         '關於六畜: 損\n'
         '關於尋人: 見\n'
         '關於公訟: 虧\n'
         '關於移徙: 守舊\n'
         '關於失物: 凶\n'
         '關於疾病: 欠安\n'
         '關於山墳: 宜改\n'
         '\n'
         '潛在的指示為地支「丑」, 也可給求籤者做相關提醒。\n'
         ' - 地理位置代表東北\n'
         ' - 時間為凌晨1點至3點\n'
         ' - 十二生肖對應為牛, 勤奮、穩重, 適合從事需要耐力和毅力的工作\n'
         ' - 五行為「土」, 土生金但土剋水, 若為上籤宜土地耕作, 建築設計, 學習歷史, 穩定工作, 家庭聚會等, '
         '若為下籤則注意過度勞累, 土石傷害, 過度保守, 環境破壞等\n'
         '\n'
         '\n'
         'Guidelines: - Interpret the provided [Context] flexibly. The '
         '"籤詩解讀" should connect the content of the divination with the '
         'user\'s question. - Use the "小廟公建議" to provide specific, '
         'actionable guidance relevant to the question, referencing '
         'details from the context where appropriate. - End with a short '
         'phrase encouraging the user, alike "Don\'t worry,", "Good luck", '
         'etc. \n'
         '\n'
         'Note: If the length of the [Context] is less than 20 characters, '
         'provide the full [Context] directly as a response without '
         'applying the [Response Format].')},
    # {'role': 'user', 'content': '整體運勢',}
    {'role': 'user', 'content': '最近身體欠佳, 請給予指示', }
]


def save_response(file_path, data):
    """Saves or appends llm response to a JSON file"""

    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
            existing_data.append(data)  # Add new entry to existing data
    else:
        # If file doesn't exist, initialize data with the first entry.
        existing_data = [data]

    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, indent=4, ensure_ascii=False)


system_prompt = [
    prompt for prompt in context_list if prompt['role'] == 'system'][0]['content']
user_prompt = [
    prompt for prompt in context_list if prompt['role'] == 'user'][0]['content']
user_messages = [prompt for prompt in context_list if prompt['role'] == 'user']

print(f'system: {system_prompt[:10]}...')
print(f'user: {user_prompt}...')
print(f'messages: {user_messages}...')

system: You are an...
user: 最近身體欠佳, 請給予指示...
messages: [{'role': 'user', 'content': '最近身體欠佳, 請給予指示'}]...
time: 9.96 ms (started: 2024-12-17 03:57:04 +00:00)


## openai


In [3]:
# Protect your sensitive API_KEY, goto colab > secrets > add key
import os

from google.colab import userdata
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_key_nycu')
openai_client = OpenAI(
    api_key=os.environ.get('OPENAI_API_KEY'),
)

time: 7.13 s (started: 2024-12-17 03:57:04 +00:00)


In [ ]:
import datetime
import timeit
from typing import Optional

openai_model_list = ['gpt-4o', 'gpt-4o-mini']
max_tokens_list = [None, 512, 1024, 2048]
temperature_list = [0.25, 0.5, 0.75, 0.9]

openai_messages = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt}
]


def openai_api_call(
    model_name: str,
    temperature: int,
    max_tokens: Optional[int] = None
):
    openai_response = openai_client.chat.completions.create(
        model=model_name,
        max_tokens=max_tokens,
        messages=openai_messages,
        temperature=temperature,
    )
    return openai_response


for model_name in openai_model_list:
    for max_tokens in max_tokens_list:
        for temperature in temperature_list:
            start_time = timeit.default_timer()
            openai_response = openai_api_call(
                model_name=model_name,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            end_time = timeit.default_timer()

            print(
                f'model: {model_name}, temperature: {temperature}, max_tokens:{max_tokens}\n{"="*20}')
            print('message:', openai_response.choices[0].message.content)
            print('time:', end_time - start_time)
            print()

            # Save
            time_diff = end_time - start_time
            time_diff_obj = datetime.timedelta(seconds=time_diff)
            human_readable_time = str(time_diff_obj)
            save_data = {
                "model": model_name,
                "temperature": temperature,
                "max_tokens": max_tokens,
                'system': system_prompt,
                'messages': openai_messages,
                "response_text": openai_response.choices[0].message.content.strip(),
                "execution_time": human_readable_time
            }
            save_response('chatgpt_response_v2.json', save_data)

model: gpt-4o, temperature: 0.25, max_tokens:None
message: 您抽到的籤是: 【中籤】中中 第五十八首-羅隱歸咎越王  
籤詩: 忠言善語君須記 莫向他方求別藝 勸君安守舊生涯 除卻有餘都不是  

籤詩解讀:  
這首籤詩提醒您要謹記忠言善語，尤其在健康方面，應該聽取專業醫生的建議，不要輕易嘗試不熟悉的療法或偏方。當前的狀況建議您保持現有的生活方式，避免過度勞累或嘗試新的激烈活動，以免加重身體負擔。

小廟公建議:  
- 關於自身: 目前健康狀況不利，建議您多休息，並定期檢查身體狀況。
- 關於疾病: 欠安，應該及早就醫，並遵循醫囑。
- 地支「丑」的指示提醒您，像牛一樣勤奮但穩重，適合從事需要耐力和毅力的工作，但在健康方面，應避免過度勞累。

請保持樂觀，並相信在專業的指導下，您的健康會逐漸好轉。祝您早日康復！
time: 3.619814163999763

model: gpt-4o, temperature: 0.5, max_tokens:None
message: 您抽到的籤是: 【中籤】中中 第五十八首-羅隱歸咎越王  
籤詩: 忠言善語君須記 莫向他方求別藝 勸君安守舊生涯 除卻有餘都不是

籤詩解讀:
這首籤詩提醒您要謹記忠言和善意的建議，特別是在健康方面，不要輕易嘗試新的療法或偏方。當下最重要的是保持現有的生活方式，避免過度操勞和不必要的壓力。這段時間，請多關注身邊親友的建議，他們的關心可能會帶來有益的啟示。

小廟公建議:
- 關於自身: 此時對您的健康不利，建議您多加留意身體狀況，及早就醫，並遵循醫生的專業建議。
- 關於疾病: 目前健康狀況可能不太穩定，建議您保持規律的作息，適當休息，並多攝取營養。
- 地支「丑」提醒您，適合從事需要耐力和毅力的活動，但要避免過度勞累。

請務必保重身體，保持樂觀心態，祝您早日康復！
time: 3.3523421629997756

model: gpt-4o, temperature: 0.75, max_tokens:None
message: 您抽到的籤是: 【中籤】中中 第五十八首-羅隱歸咎越王  
籤詩: 忠言善語君須記 莫向他方求別藝 勸君安守舊生涯 除卻有餘都不是  

籤詩解讀:  
這支籤詩告訴您要保持謙虛，聽取忠言和善意的建議

## anthropic

- models https://docs.anthropic.com/en/docs/about-claude/models
- check limits https://console.anthropic.com/settings/limits

| MODEL                                             | REQUESTS PER MINUTE | INPUT TOKENS PER MINUTE | OUTPUT TOKENS PER MINUTE |
| ------------------------------------------------- | ------------------- | ----------------------- | ------------------------ |
| Claude 3.5 Sonnet 2024-10-22                      | 5                   | 20,000                  | 4,000                    |
| Claude 3.5 Sonnet 2024-06-20                      | 5                   | 20,000                  | 4,000                    |
| Claude 3.5 Haiku                                  | 5                   | 25,000                  | 5,000                    |
| Claude 3 Opus                                     | 5                   | 10,000                  | 2,000                    |
| Claude 3 Sonnet                                   | 5                   | 20,000                  | 4,000                    |
| Claude 3 Haiku                                    | 5                   | 25,000                  | 5,000                    |
| Batch requests Limit per minute across all models | -                   | -                       | -                        |

model IDs for each of the models listed

| MODEL                        | MODEL ID                      |
| ---------------------------- | ----------------------------- |
| Claude 3.5 Sonnet 2024-10-22 | claude-v3.5-sonnet-2024-10-22 |
| Claude 3.5 Sonnet 2024-06-20 | claude-v3.5-sonnet-2024-06-20 |
| Claude 3.5 Haiku             | claude-v3.5-haiku             |
| Claude 3 Opus                | claude-v3-opus                |
| Claude 3 Sonnet              | claude-v3-sonnet              |
| Claude 3 Haiku               | claude-v3-haiku               |


In [ ]:
# !pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
# Protect your sensitive API_KEY, goto colab > secrets > add key
import anthropic
from google.colab import userdata

claude_api_key = userdata.get('ANTHROPIC_API_KEY')
anthropic_client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=claude_api_key,
)

time: 2.03 s (started: 2024-12-15 04:21:02 +00:00)


In [ ]:
import datetime
import timeit
from typing import Optional

claude_model_list = ["claude-3-5-sonnet-20241022", 'claude-3-opus-20240229']
max_tokens_list = [512, 1024, 2048]
temperature_list = [0.25, 0.5, 0.75, 0.9]

system_prompt = [
    prompt for prompt in context_list if prompt['role'] == 'system'][0]['content']
anthropic_messages = [
    prompt for prompt in context_list if prompt['role'] == 'user']


def anthropic_api_call(
    model_name: str,
    temperature: float,
    max_tokens: Optional[int] = None,
):
    claude_message = anthropic_client.messages.create(
        model=model_name,
        max_tokens=max_tokens,
        # messages=[{"role": "user", "content": "Hello, Claude"}]
        system=system_prompt,
        messages=anthropic_messages,
        temperature=temperature,
    )
    return claude_message


for model_name in claude_model_list:
    for max_tokens in max_tokens_list:
        for temperature in temperature_list:
            start_time = timeit.default_timer()
            anthropic_response = anthropic_api_call(
                model_name=model_name,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            end_time = timeit.default_timer()

            print(
                f'model_name: {model_name}, temperature: {temperature}, max_tokens:{max_tokens}\n{"="*20}')
            print('message:', anthropic_response.content[0].text)
            print('time:', end_time - start_time)
            print()

            # Save
            time_diff = end_time - start_time
            time_diff_obj = datetime.timedelta(seconds=time_diff)
            human_readable_time = str(time_diff_obj)
            save_data = {
                "model": model_name,
                "temperature": temperature,
                "max_tokens": max_tokens,
                'system': system_prompt,
                'messages': anthropic_messages,
                "response_text": anthropic_response.content[0].text.strip(),
                "execution_time": human_readable_time
            }
            save_response('claude_response_v2.json', save_data)

model_name: claude-3-5-sonnet-20241022, temperature: 0.25, max_tokens:512
message: 您抽到的籤是: 【中籤】中中 第五十八首-羅隱歸咎越王

籤詩:
忠言善語君須記     
莫向他方求別藝     
勸君安守舊生涯     
除卻有餘都不是   

籤詩解讀:
這支籤顯示您目前的身體狀況確實需要多加注意。籤詩中提到「安守舊生涯」，暗示您可能因為生活作息不規律或過度操勞而影響健康。「忠言善語君須記」也提醒您要認真聽取醫生或親友對您健康的建議，不要固執己見。籤詩中的「莫向他方求別藝」，建議您不要過於相信偏方或是未經證實的療法。

小廟公建議:
1. 籤詩顯示「疾病欠安」，建議及早就醫診治，遵醫囑用藥
2. 特別注意凌晨1-3點的休息，這是身體重要的修復時段
3. 避免過度勞累，適當調整作息，保持規律生活
4. 由於地支屬「土」，要特別注意腸胃功能，飲食要清淡均衡
5. 可以在東北方向放些平安符物，有助於改善運勢
6. 不建議嘗試未經驗證的偏方或另類療法
7. 保持樂觀正向的心態，但也要實事求是地面對身體狀況

放心，只要按部就班調養，保持耐心，您的身體狀況會逐漸改善的。祝您早日康復！
time: 8.437424507999822

model_name: claude-3-5-sonnet-20241022, temperature: 0.5, max_tokens:512
message: 您抽到的籤是: 【中籤】中中 第五十八首-羅隱歸咎越王

籤詩:
忠言善語君須記     
莫向他方求別藝     
勸君安守舊生涯     
除卻有餘都不是  

籤詩解讀:
這支籤顯示您目前的身體狀況確實需要多加注意。籤詩中提到「安守舊生涯」，暗示您可能因為生活作息不規律或過度操勞而影響健康。籤象顯示「疾病：欠安」，建議您應該重視身體發出的警訊。另外，「忠言善語君須記」提醒您要多聽取醫生或親友對健康方面的建議。

小廟公建議:
1. 建議在凌晨1-3點（丑時）避免熬夜，這段時間應該好好休息
2. 由於籤詩暗示「土」的特質，提醒您：
   - 避免過度勞累
   - 注意腸胃健康
   - 保持規律作息
3. 不建議嘗試偏方或另類療法，應該遵循正規醫療建議
4. 可以在東北方向放置一些平安符號或

## aisuite


In [ ]:
import os

# Protect your sensitive API_KEY, goto colab > secrets > add key
from google.colab import userdata

# export OPENAI_API_KEY="your-openai-api-key"
# export ANTHROPIC_API_KEY="your-anthropic-api-key"
os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_key_nycu')
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

In [ ]:
import aisuite as ai


def _create_provider(cls, provider_key, config):  # Add back 'config' argument
    # Remove 'proxies' from config if present
    config.pop("proxies", None)
    # Instantiate the provider class
    provider_class = getattr(ai.providers, provider_key)
    return provider_class(**config)


# Replace the original method
ai.provider.ProviderFactory.create_provider = _create_provider

In [ ]:
suite_client = ai.Client()

suite_models = ["openai:gpt-4o", "anthropic:claude-3-5-sonnet-20240620"]
temperature = TEMPERATURE  # 0.75

for model in suite_models:
    try:
        suite_response = suite_client.chat.completions.create(
            model=model,
            # messages=messages,
            messages=context_list,
            temperature=temperature,
        )
        print(f'model: {model}\n{"="*20}')
        print(suite_response.choices[0].message.content)
    except Exception as e:
        print(f'Error in {model}: {e}')
        continue

Error in openai:gpt-4o: _create_provider() missing 1 required positional argument: 'config'
Error in anthropic:claude-3-5-sonnet-20240620: _create_provider() missing 1 required positional argument: 'config'


# 2.Comparison


### Claude-3.5-sonnet-20241022 vs. Claude-3-opus-20240229

| Feature                   | Claude-3.5-sonnet-20241022                                                                     | Claude-3-opus-20240229                                                             |
| ------------------------- | ---------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------- |
| **Overall Capability**    | More advanced, with improved factual accuracy and reasoning abilities.                         | Powerful model for complex tasks, but less advanced than Claude-3.5-sonnet.        |
| **Context Window**        | 200K tokens                                                                                    | 200K tokens                                                                        |
| **Maximum Output Tokens** | 4,096 tokens                                                                                   | 4,096 tokens                                                                       |
| **Knowledge Cutoff**      | April 2024                                                                                     | August 2023                                                                        |
| **Cost**                  | More expensive                                                                                 | Less expensive                                                                     |
| **Best Use Cases**        | Complex tasks requiring factual accuracy and reasoning, such as research, writing, and coding. | General-purpose tasks, including creative writing, summarization, and translation. |


---

### **OpenAI Model Background Information**

| **Model**                     | **Model Size**     | **Number of Parameters** | **Training Data Scale**   | **Primary Use Cases**                                                                 |
|--------------------------------|--------------------|--------------------------|---------------------------|---------------------------------------------------------------------------------------|
| **gpt-4o**                     | Large             | ~175 billion parameters  | Several TB of text data   | Suitable for a wide range of language tasks, including complex dialogues, creative writing, and specialized tasks. |
| **gpt-4o-mini**                | Medium            | ~10 billion parameters   | Several hundred GB of text data | Suitable for general-purpose language tasks such as summarization, Q&A, and basic creative writing, but with lower performance compared to `gpt-4o`. |

---

### **Summary of Key Differences**

| **Model**       | **Strengths**                                                                      | **Weaknesses**                                                        |
| --------------- | ---------------------------------------------------------------------------------- | --------------------------------------------------------------------- |
| **gpt-4o**      | Strong in a wide range of tasks; excellent context understanding; highly creative. | May struggle with highly specialized tasks; slower for complex tasks. |
| **gpt-4o-mini** | Faster response time; more efficient token usage; suitable for basic tasks.        | Limited performance in complex or specialized tasks; less creative.   |
